### Общий план:
- __Заполнить пропуски в данных диагностики__
- __Агрегировать результаты диагностики по каждому юзеру__
- Выделить фичи из текста
- Агрегировать фичи по юзеру
- Объединить датасеты
- Построить матрицу корреляций по полученному датасету
- Наиболее удачные, яркие корреляции рассмотреть подробнее
- Построить бейзлайн, лр/бустинг, для прогнозирования результата диагностики по фичам текста

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

## Данные диагностик

In [2]:
train_df = []
test_df = []

for i in range(1,8):
    df = pd.read_excel('database0{}.xlsx'.format(i), header=1)
    train_df.append(df)

for i in range(8,10):
    df = pd.read_excel('database0{}.xlsx'.format(i), header=1)
    test_df.append(df)

df = pd.read_excel('database10.xlsx', header=1)    
test_df.append(df)    
    
train = pd.concat(train_df)
test = pd.concat(test_df)

In [3]:
train.shape, test.shape

((2517, 137), (1154, 137))

In [4]:
train.columns

Index(['Unti ID', 'Прогресс прохождения обязательных активностей',
       'Количество пройденных обязательных активностей',
       'Количество пройденных дополнительных активностей',
       'Прогресс прохождения обязательных активностей.1',
       'Количество пройденных обязательных активностей.1',
       'Количество пройденных дополнительных активностей.1',
       'толерантность, выходные интенсивов 06.2020',
       'Большие данные (выходные)', 'Нейротехнологии, VR и AR (выходные)',
       ...
       'Количество кликов, которые принесли какие-либо баллы за всю игру (1 минута)',
       'Количество кликов на цифры за всю игру',
       'Количество кликов на иероглифы за всю игру',
       'Количество кликов на буквы за всю игру',
       'Количество кликов на любой символ на голубом фоне',
       'Количество кликов на любой символ на желтом фоне',
       'Количество кликов на любой символ на зеленом фоне',
       'Время каждого клика (в секундах от начала игры)',
       'Среднее арифметиче

In [5]:
len(train['Unti ID'].unique()), len(test['Unti ID'].unique())

(2460, 1137)

In [6]:
len(set(test['Unti ID']) - set(train['Unti ID']))

1122

## Текст рефлексий

In [7]:
texts = pd.read_excel('Рефлексии_участников_задача_по_диагностике.xlsx')

In [8]:
texts.shape

(26546, 15)

In [9]:
len(texts['untiID'].unique())

450

In [10]:
texts.isna().sum()

eventUUID                 0
eventTitle            14038
a_title                   0
startDate                 0
startTime                 0
endDate                   0
endTime                   0
untiID                    0
feedbackQuestionID        0
question                  0
value                  8174
data                  26063
Unnamed: 12           26504
Unnamed: 13           26543
Unnamed: 14           26544
dtype: int64

In [11]:
train.shape, test.shape

((2517, 137), (1154, 137))

In [12]:
len(train['Unti ID'].unique())

2460

In [13]:
print("Кол-во уникальных ID в трейне, для к-х есть текстовые данные: {}".format(len(train['Unti ID'].unique()) - len(set(train['Unti ID'].unique()) - set(texts['untiID'].unique()))))
print("Кол-во уникальных ID в тесте, для к-х есть текстовые данные: {}".format(len(test['Unti ID'].unique()) - len(set(test['Unti ID'].unique()) - set(texts['untiID'].unique()))))



Кол-во уникальных ID в трейне, для к-х есть текстовые данные: 206
Кол-во уникальных ID в тесте, для к-х есть текстовые данные: 127


In [14]:
# ID-шники для которых нет текстов рефлексий
len(set(train['Unti ID'].unique()) - set(texts['untiID'].unique()))

2254

In [15]:
# ID-шники для которых нет результатов диагностик
len(set(texts['untiID'].unique()) - set(train['Unti ID'].unique()))
# 450 unique text id - 244 = 206

244

In [16]:
# готовим трейн - берем id только тех у кого есть текстовые данные

train_ids = set(train['Unti ID']) - (set(train['Unti ID'].unique()) - set(texts['untiID'].unique()))
test_ids = set(test['Unti ID']) - (set(test['Unti ID'].unique()) - set(texts['untiID'].unique()))

In [17]:
df_train = train[train['Unti ID'].isin(train_ids)]
df_train.shape

(225, 137)

In [18]:
df_test = test[test['Unti ID'].isin(test_ids)]
df_test.shape

(134, 137)

### Заполнить пропуски в данных диагностики
Подход 1: заполнить нулем

In [20]:
len(df_train['Unti ID'].unique())

206

In [21]:
df_train[df_train['Unti ID']==103]

,Unti ID,Прогресс прохождения обязательных активностей,Количество пройденных обязательных активностей,Количество пройденных дополнительных активностей,Прогресс прохождения обязательных активностей.1,Количество пройденных обязательных активностей.1,Количество пройденных дополнительных активностей.1,"толерантность, выходные интенсивов 06.2020",Большие данные (выходные),"Нейротехнологии, VR и AR (выходные)",...,"Количество кликов, которые принесли какие-либо баллы за всю игру (1 минута)",Количество кликов на цифры за всю игру,Количество кликов на иероглифы за всю игру,Количество кликов на буквы за всю игру,Количество кликов на любой символ на голубом фоне,Количество кликов на любой символ на желтом фоне,Количество кликов на любой символ на зеленом фоне,Время каждого клика (в секундах от начала игры),"Среднее арифметическое времени между сменой правила и кликом, приносящим 1 или 3 балла.",Среднее арифметическое времени между сменой правила и первым кликом внутри этого правила (вне зависимости от правильности клика)
3,103,20.0,1.0,2.0,0.0,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,103,17.0,1.0,2.0,0.0,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,103,17.0,1.0,2.0,0.0,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,103,20.0,1.0,2.0,0.0,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,103,20.0,1.0,2.0,0.0,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,103,17.0,1.0,2.0,0.0,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,103,20.0,1.0,2.0,0.0,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
df_train.select_dtypes(include='object').isna().sum()

Время, проведенное на экране раунда 2 (сложность 1)                           169
Время, проведенное на экране раунда 3 (сложность 1)                           169
Время, проведенное на экране раунда 1 (сложность 2)                           190
Время, проведенное на экране раунда 2 (сложность 2)                           190
Количество кликов (вне зависимости от их правильности) для каждого правила    222
Количество кликов внутри каждого правила, которые принесли какие-то баллы     222
Время каждого клика (в секундах от начала игры)                               222
dtype: int64

In [23]:
# convert to numeric
df_train = df_train.apply(pd.to_numeric, errors='coerce')
df_test = df_test.apply(pd.to_numeric, errors='coerce')

In [24]:
df_train_num = df_train.fillna(0)
df_test_num = df_test.fillna(0)

sum(df_train_num.isna().sum()), sum(df_test_num.isna().sum())

(0, 0)

### Агрегировать результаты диагностики по каждому юзеру

In [25]:
from functools import reduce

def aggregate(data):
    sum_df = data.groupby(['Unti ID'], as_index=False).agg(np.sum)
    sum_df.columns = [str(col) + '_sum' if col != 'Unti ID' else 'Unti ID' for col in sum_df.columns]
    
    mean_df = data.groupby(['Unti ID'], as_index=False).agg(np.mean)
    mean_df.columns = [str(col) + '_mean' if col != 'Unti ID' else 'Unti ID' for col in mean_df.columns]
    
    median_df = data.groupby(['Unti ID'], as_index=False).agg(np.median)
    median_df.columns = [str(col) + '_median' if col != 'Unti ID' else 'Unti ID' for col in median_df.columns]
    
    max_df = data.groupby(['Unti ID'], as_index=False).agg(np.max)
    max_df.columns = [str(col) + '_max' if col != 'Unti ID' else 'Unti ID' for col in max_df.columns]
    
    min_df = data.groupby(['Unti ID'], as_index=False).agg(np.min)
    min_df.columns = [str(col) + '_min' if col != 'Unti ID' else 'Unti ID' for col in min_df.columns]
    
    data_frames = [sum_df, mean_df, median_df, max_df, min_df]
#     df_merged = pd.merge(sum_df, mean_df, on=['Unti ID'])
    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Unti ID'], how='right'), data_frames)
    return df_merged

In [26]:
df_train_agg = aggregate(df_train_num)
df_train_agg.shape

(206, 681)

In [27]:
# sanity check - all good
for col in df_train_agg.columns:
    if 'Большие данные (выходные)' in col:
        print(col)

Большие данные (выходные)_sum
Большие данные (выходные)_mean
Большие данные (выходные)_median
Большие данные (выходные)_max
Большие данные (выходные)_min


In [28]:
df_test_agg = aggregate(df_test_num)
df_test_agg.shape

(127, 681)

In [29]:
df_train_agg.rename(columns={'Unti ID':'untiID'}, inplace=True)
df_test_agg.rename(columns={'Unti ID':'untiID'}, inplace=True)

In [38]:
# Сохраним сформированные трейн и тест
df_train_agg.to_csv('diag_train_agg.csv',index=False)
df_test_agg.to_csv('diag_test_agg.csv',index=False)

### Фикс бага в данных рефлексий

In [30]:
# Странные id вопросов
texts['feedbackQuestionID'].unique()

array([44, 46, 43, 55, 45, 'Воробьев Роман Дмитриевич',
       'Селина Алина Валерьевна', 'Новиков Алекcандр Андреевич',
       'Анисимов Илья Андреевич', 32, 33, 58, 59, 60, 50, 51, 52, 53, 54,
       62, 63, 64, 65, 66, 67, 85, 86, 87, 88, 89, 90, 91, 92],
      dtype=object)

In [32]:
texts[texts['feedbackQuestionID']=='Селина Алина Валерьевна'].head(3)

,eventUUID,eventTitle,a_title,startDate,startTime,endDate,endTime,untiID,feedbackQuestionID,question,value,data,Unnamed: 12,Unnamed: 13,Unnamed: 14
7250,a0e46d80-9ab0-43a1-bcf6-0b1e26bf4516,NaN,Визионерская среда: просмотр и обсуждение виде...,«Миф о власти искусственного интеллекта: персп...,2020-05-06,15:00:00,2020-05-06,710992,Селина Алина Валерьевна,44,"Насколько вероятно, что Вы порекомендуете меро...",8,2020-05-06 16:17:35,NaN,NaN
7251,a0e46d80-9ab0-43a1-bcf6-0b1e26bf4516,NaN,Визионерская среда: просмотр и обсуждение виде...,«Миф о власти искусственного интеллекта: персп...,2020-05-06,15:00:00,2020-05-06,710992,Селина Алина Валерьевна,46,"Перечислите основные этапы деятельности, в кот...","Сначала изучали математику нейронные сетей, ме...",2020-05-06 16:17:35,NaN,NaN
7252,a0e46d80-9ab0-43a1-bcf6-0b1e26bf4516,NaN,Визионерская среда: просмотр и обсуждение виде...,«Миф о власти искусственного интеллекта: персп...,2020-05-06,15:00:00,2020-05-06,710992,Селина Алина Валерьевна,43,"Чему новому, как вам кажется, вы научились?","Узнала очень много нового о нейронных сетях, т...",2020-05-06 16:17:35,NaN,NaN


#### Похоже, что для некоторых сэмплов значения колонок сместились. Вернем id вопросов, вопросы и ответы в соответствующие колонки

In [33]:
ids_to_fix = []
for val in texts['feedbackQuestionID'].unique():
    if not isinstance(val, int):
        ids_to_fix.append(texts[texts['feedbackQuestionID']==val]['untiID'].unique()[0])

In [34]:
def fix_strange_ids(ids):
    for i in ids:
        texts['feedbackQuestionID'] = np.where(texts.untiID == i, texts['question'], texts['feedbackQuestionID'])
        texts['question'] = np.where(texts.untiID == i, texts['value'], texts['question'])
        texts['value'] = np.where(texts.untiID == i, texts['data'], texts['value'])
    return texts

In [35]:
texts.shape

(26546, 15)

In [36]:
text_df = fix_strange_ids(ids_to_fix)

In [37]:
# sanity check - all good
text_df['feedbackQuestionID'].unique()

array([44, 46, 43, 55, 45, 32, 33, 58, 59, 60, 50, 51, 52, 53, 54, 62, 63,
       64, 65, 66, 67, 85, 86, 87, 88, 89, 90, 91, 92], dtype=object)

In [41]:
text_df.shape, text_df.drop_duplicates().shape

((26546, 15), (9128, 15))

In [42]:
# Убираем дубликаты
text_df.drop_duplicates(inplace=True)
text_df.shape

(9128, 15)

In [43]:
# Сохраним исправленные данные рефлексий
text_df.to_csv('fixed_reflex_texts.csv',index=False)